<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [ ]:
# Install spark

In [1]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

In [2]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

#import functions/Classes for sparkml

from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

# import functions/Classes for metrics
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
# Start session

In [4]:
spark = SparkSession.builder.appName("capstone_project").getOrCreate()

23/09/28 10:41:14 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [5]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

--2023-09-28 10:41:40--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv’

searchterms.csv     100%[===================>] 227.99K  --.-KB/s    in 0.005s  

2023-09-28 10:41:40 (40.8 MB/s) - ‘searchterms.csv’ saved [233457/233457]



In [9]:
# Load the csv into a spark dataframe

In [22]:
df = spark.read.csv("searchterms.csv", header=True, inferSchema=True)

In [23]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [24]:
print("nb_rows = ", df.count())
print("nb_columns = ", len(df.columns))

nb_rows =  10000
nb_columns =  4


In [25]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [26]:
df.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [27]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [28]:
type(df.searchterm)

pyspark.sql.column.Column

In [ ]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [29]:
df.groupby("searchterm").count().show()

[Stage 27:============================================>          (60 + 13) / 75]

+-------------------+-----+
|         searchterm|count|
+-------------------+-----+
|          mobile 5g| 2301|
|ebooks data science|  410|
|      mobile 6 inch| 2312|
|     tablet 10 inch|  715|
|             laptop|  935|
|      mobile latest| 1327|
|      gaming laptop|  499|
|     laptop 14 inch|  461|
|        tablet wifi|  896|
|          pen drive|  144|
+-------------------+-----+



In [62]:
df.groupby("year").count().show()

+----+-----+
|year|count|
+----+-----+
|2021|10000|
+----+-----+



In [ ]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [54]:
from pyspark.sql.functions import desc

df.groupby("searchterm")\
.count()\
.sort(desc("count"))\
.show(5)

[Stage 43:================================================>    (184 + 10) / 200]

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+
only showing top 5 rows



In [ ]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

In [ ]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [31]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

--2023-09-28 10:51:54--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz’

model.tar.gz        100%[===================>]   1.46K  --.-KB/s    in 0s      

2023-09-28 10:51:54 (12.4 MB/s) - ‘model.tar.gz’ saved [1490/1490]



In [66]:
!tar -xzvf model.tar.gz

sales_prediction.model/
sales_prediction.model/metadata/
sales_prediction.model/metadata/part-00000
sales_prediction.model/metadata/.part-00000.crc
sales_prediction.model/metadata/_SUCCESS
sales_prediction.model/metadata/._SUCCESS.crc
sales_prediction.model/data/
sales_prediction.model/data/part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet
sales_prediction.model/data/_SUCCESS
sales_prediction.model/data/.part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet.crc
sales_prediction.model/data/._SUCCESS.crc


In [75]:
from pyspark.ml.regression import LinearRegressionModel

In [74]:
loaded_model = LinearRegressionModel.load("./sales_prediction.model/")

In [79]:
loaded_data = spark.read.parquet("./sales_prediction.model/data/part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet")

In [ ]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [89]:
testing_data = df[df.year == 2021]
testing_data = testing_data[["day", "month", "year"]]

In [90]:
assembler = VectorAssembler(inputCols=testing_data.columns, outputCol="featurs")

In [91]:
transformed_data = assembler.transform(testing_data)

In [92]:
testing_data.show(5)

+---+-----+----+
|day|month|year|
+---+-----+----+
| 12|   11|2021|
| 12|   11|2021|
| 12|   11|2021|
| 12|   11|2021|
| 12|   11|2021|
+---+-----+----+
only showing top 5 rows



In [93]:
predictions = loaded_model.transform(testing_data)

IllegalArgumentException: 'Field "features" does not exist.\nAvailable fields: day, month, year'